In [109]:
from qiskit import QuantumRegister, QuantumCircuit
from qiskit import Aer, transpile
from qiskit.quantum_info import random_clifford

In [110]:
n_qubits = 20
T = 10
qr = QuantumRegister(n_qubits, 'q')
circ = QuantumCircuit(qr)

for t in range(T):
    for i in range(n_qubits // 2):
        circ.append(random_clifford(2), [qr[2*i], qr[2*i+1]])
    for j in range((n_qubits - 1) // 2):
        circ.append(random_clifford(2), [qr[2*j+1], qr[2*j+2]])

circ.measure_all()

In [111]:
# circ.draw('mpl')

In [112]:
simulator = Aer.get_backend('aer_simulator_stabilizer')
circ = transpile(circ, simulator)

In [113]:
# circ.draw('mpl')

In [125]:
result = simulator.run(circ, shots = 100000).result()

In [126]:
result.get_counts()

{'00110010110001011010': 1,
 '00111000011111110010': 1,
 '01001110000101010001': 1,
 '00001100011101001101': 1,
 '10110101011011010111': 1,
 '10101100101011100110': 1,
 '00111001111001110100': 1,
 '00111101000011010011': 1,
 '01110100100110100001': 1,
 '11111011101111111000': 1,
 '00100100110101001111': 1,
 '11011001000110111000': 1,
 '11110001101110000111': 1,
 '00110111101100000000': 1,
 '11011011101111100111': 1,
 '10011100010110101111': 1,
 '11101010001101100111': 1,
 '01011010000101000011': 1,
 '01101011110000010000': 1,
 '10110110111001001110': 1,
 '00101100001000100011': 1,
 '00110001001111001011': 1,
 '00011100001000000010': 1,
 '00010101011001100110': 1,
 '11001010111001101100': 1,
 '11001111000001101100': 1,
 '10111000111101111001': 1,
 '11100000011001010000': 1,
 '00000011111100111000': 1,
 '00000111100010101101': 1,
 '01101101100010111000': 1,
 '01011100111101101101': 1,
 '10010010011000001010': 1,
 '10111110000000000101': 1,
 '00011111101000001011': 1,
 '001010101111101100